In [ ]:
!pip install -q smolagents
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

In [ ]:
# model_id = "meta-llama/Llama-3.3-70B-Instruct "

In [ ]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())  # (model_id=model_id))

In [ ]:
agent

In [ ]:
agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

In [ ]:
agent.run("How does DeepSeek-v3 model size compares to DeepSeek-v2? What is the percent increase or decrease from one version to the other? ")

In [ ]:
agent.run("How does DeepSeek-v3 model size compares to DeepSeek-v2? What is the percent increase or decrease from one version to the other? How does it compare to llama-3.3?")

In [ ]:
671 / 160

# Retrieval Example

In [ ]:
!pip install smolagents pandas langchain langchain-community sentence-transformers faiss-cpu --upgrade -q

In [ ]:
!pip install datasets -q
!pip install rank_bm25 -q

### Problems with Vanilla RAG

- It performs only one retrieval step: if the results are bad, the generation in turn will be bad.
- Semantic similarity is computed with the user query as a reference, which might be suboptimal: for instance, the user query will often be a question and the document containing the true answer will be in affirmative voice, so its similarity score will be downgraded compared to other source documents in the interrogative form, leading to a risk of missing the relevant information.

In [ ]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

In [ ]:
docs_processed

In [ ]:
print(docs_processed[0])

In [ ]:
# how to build tools - https://huggingface.co/docs/smolagents/tutorials/tools

from smolagents import Tool


class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(docs, k=10)

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(query,)
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )


retriever_tool = RetrieverTool(docs_processed)

In [ ]:
retriever_tool.forward("How to download a model from Huggingface?")

In [ ]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(
    tools=[retriever_tool], model=HfApiModel(), max_iterations=4, verbose=True
)

In [ ]:
agent.run("How to download a model from Huggingface?")

In [ ]:
agent_output = agent.run("For a transformers model training, which is slower, the forward or the backward pass?")

print("Final output:")
print(agent_output)

In [ ]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(
    tools=[retriever_tool, DuckDuckGoSearchTool], model=HfApiModel(), max_iterations=4, verbose=True
)

In [ ]:
agent.run("How to download a model from Huggingface?")